In [ ]:
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk
from bigdata_a3_utils import download_all_amazon_reviews, load_compressed_dataset
import polars as pl

In [ ]:
# auto_review: Dataset = load_dataset(
#     "McAuley-Lab/Amazon-Reviews-2023",
#     "raw_review_Automotive",
#     split="full",
#     trust_remote_code=True,
# )

# auto_meta: Dataset = load_dataset(
#     "McAuley-Lab/Amazon-Reviews-2023",
#     "raw_meta_Automotive",
#     split="full",
#     trust_remote_code=True,
# )

In [ ]:
# auto_review.to_parquet("amazon_review_auto.parquet")
# auto_meta.to_parquet("amazon_meta_auto.parquet")

In [ ]:
df_review: pl.DataFrame = pl.read_parquet("amazon_review_auto.parquet")
df_meta: pl.DataFrame = pl.read_parquet("amazon_meta_auto.parquet")

In [ ]:
def extract_brand() -> pl.Expr:
    store_clean: pl.Then = pl.when(pl.col("store").str.strip_chars() != "").then(pl.col("store"))
    details_brand: pl.Expr = pl.col("details").str.extract(r"Brand[:\s]*([^\s,;]+)", 1)
    
    return pl.coalesce([
        store_clean,
        details_brand,
        pl.lit("Unknown")
    ]).alias("brand")

In [ ]:
# Drop invalid ratings (missing or not in [1,2,3,4,5])
df_review: pl.DataFrame = df_review.filter(pl.col("rating").is_in([1, 2, 3, 4, 5]))
# Drop empty review texts
df_review = df_review.filter(pl.col("text").str.strip_chars().str.len_chars() > 0)

df_review = df_review.with_columns([
    pl.col("text").str.count_matches(r"\b\w+\b").alias("review_length"),
    (pl.col("timestamp").cast(pl.Datetime("ms")).dt.year()).alias("year")
])

In [ ]:
df: pl.DataFrame = df_review.join(df_meta, on="parent_asin", how="left")

In [ ]:
df: pl.DataFrame = df.with_columns([extract_brand()])
df = df.unique(subset=["user_id", "text", "asin"], keep="first")

In [ ]:
df

In [ ]:
# # Drop invalid ratings (missing or not in [1,2,3,4,5])
# df: pl.DataFrame = df.filter(pl.col("rating").is_in([1, 2, 3, 4, 5]))
# # Drop empty review texts
# df = df.filter(pl.col("text").str.strip_chars().str.len_chars() > 0)

# df = df.with_columns([extract_brand()])

# df = df.with_columns([
#     pl.col("text").str.count_matches(r"\b\w+\b").alias("review_length"),
#     (pl.col("timestamp").cast(pl.Datetime("ms")).dt.year()).alias("year")
# ])


# df = df.unique(subset=["user_id", "text", "asin"], keep="first")

In [ ]:
# df.write_parquet("data/intermediate/amazon_auto.parquet")